In [2]:
import pandas as pd
import os
from main import *

In [ ]:
#Load the data

In [34]:
en_pairs = ['de-en', 'ru-en', 'zh-en']

fi_df = pd.read_csv("corpus\en-fi\scores.csv")
zh_df = pd.read_csv("corpus\en-zh\scores.csv")
en_df = pd.read_csv("corpus\cs-en\scores.csv")

for pair in en_pairs:
    new_df = pd.read_csv(os.path.join('corpus',pair, 'scores.csv'))
    en_df = en_df.append(new_df, ignore_index=True)


In [35]:
en_df.iloc[20000:20005]

,source,reference,translation,z-score,avg-score,annotators
20000,"Nach dem Brexit-Referendum ist es ungewiss, ob...","After the Brexit referendum, it is uncertain w...",After the Brexit referendum it is unclear whet...,0.108148,96.0,2
20001,Das Rudern an sich ist schon eine Herausforder...,"Rurowing in itself is already a challenge, but...",Rowing in itself is a journey but that makes i...,0.332057,98.0,1
20002,"Dettori nimmt den Jubel auf, als er auf Predil...",Dettori takes the jubilee as he picks up victo...,Dettori soaks up the cheers as he brings in Pr...,-0.003806,95.0,1
20003,Durch die niedrigen Temperaturen läuft häufig ...,"Due to low temperatures, the organism of the a...","As a result of low temperatures, the systems o...",0.332057,98.0,1
20004,"Ihren Namen haben sie, weil sie scheinbar aus ...",They have their name because they seem to come...,Their name refers to the fact that they seem t...,0.220102,97.0,1


In [ ]:
# Preprocessing

In [37]:
en_df = the_preprocessor(en_df, lang="en")

77688it [03:50, 336.57it/s]
77688it [03:41, 351.43it/s]


In [38]:
en_df.head()

,source,z-score,pt,ref,1gram-precision,2gram-precision,3gram-precision,4gram-precision,1gram-recall,2gram-recall,3gram-recall,4gram-recall,wmdist,pt_len,ref_len
0,Uchopíte pak zbraň mezi své předloktí a rameno...,-0.675383,"[you, then, grasp, the, gun, between, your, fo...","[you, will, then, grab, the, weapon, between, ...",0.750000,0.315789,0.166667,0.117647,0.652174,0.272727,0.142857,0.100000,0.376218,20,23
1,"Ale je-li New York změna, pak je to také znovu...",-0.829403,"[but, if, new, york, is, change, it, is, also,...","[but, if, new, york, is, changed, then, it, 's...",0.700000,0.444444,0.375000,0.285714,0.583333,0.363636,0.300000,0.222222,0.533093,10,12
2,"Dlouho a intenzivně jsem během léta přemýšlel,...",0.803185,"[i, have, thought, long, and, hard, over, the,...","[i, have, been, thinking, over, and, over, aga...",0.636364,0.437500,0.258065,0.200000,0.724138,0.500000,0.296296,0.230769,0.411109,33,29
3,"Najdou si jiný způsob, jak někde podvádět.",0.563149,"[they, will, find, another, way, how, to, defr...","[they, find, another, way, to, cheat, somewhere]",0.555556,0.250000,0.142857,0.000000,0.714286,0.333333,0.200000,0.000000,0.489097,9,7
4,Zpráva o výměně v čele prezidentovy administra...,0.021549,"[the, news, of, the, replacement, at, the, top...","[the, report, on, the, replacement, of, the, p...",0.370370,0.153846,0.080000,0.041667,0.454545,0.190476,0.100000,0.052632,0.719761,27,22


In [40]:
fi_df = the_preprocessor(fin_df, lang="fi")

6748it [00:02, 2903.74it/s]
6748it [00:03, 2105.42it/s]


In [41]:
zh_df = the_preprocessor(zh_df, "zh")

10221it [00:51, 199.65it/s]
10221it [00:49, 207.13it/s]


In [ ]:
#Concat all ready for NN

In [42]:
total_df = en_df.append(fi_df, ignore_index=True) 

In [43]:
total_df = total_df.append(zh_df, ignore_index=True)

In [75]:
#Get the data ready for NN 

# final nan check 
total_df = total_df.replace(np.inf, np.nan)
total_df = total_df.dropna()

# data, target
target = total_df["z-score"]
X = total_df.drop(["source","z-score","pt","ref"], axis=1)


# standard scaler
from sklearn.preprocessing import StandardScaler
sscaler = StandardScaler()
X_ss = sscaler.fit_transform(X)

# train-test-split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.20, random_state=23)

#create_model
nnn = [11, 64, 128, 64, 1]
my_model = create_model(nnn)
my_model.summary()



In [98]:
history = my_model.fit(X_train, y_train, epochs=500, batch_size=16)

Epoch 1/500
4733/4733 [==============================] - 7s 2ms/step - loss: 0.7062 - mae: 0.6638
Epoch 2/500
4733/4733 [==============================] - 5s 1ms/step - loss: 0.6759 - mae: 0.6498
Epoch 3/500
4733/4733 [==============================] - 5s 1ms/step - loss: 0.6729 - mae: 0.6481
Epoch 4/500
4733/4733 [==============================] - 4s 894us/step - loss: 0.6694 - mae: 0.6459
Epoch 5/500
4733/4733 [==============================] - 5s 1ms/step - loss: 0.6704 - mae: 0.6449
Epoch 6/500
4733/4733 [==============================] - 5s 1ms/step - loss: 0.6663 - mae: 0.6442
Epoch 7/500
4733/4733 [==============================] - 5s 1ms/step - loss: 0.6663 - mae: 0.6441
Epoch 8/500
4733/4733 [==============================] - 5s 1ms/step - loss: 0.6644 - mae: 0.6428
Epoch 9/500
4733/4733 [==============================] - 6s 1ms/step - loss: 0.6640 - mae: 0.6424
Epoch 10/500
4733/4733 [==============================] - 4s 889us/step - loss: 0.6632 - mae: 0.6421
Epoch 11/500
47

KeyboardInterrupt: 

In [86]:
preds = my_model.evaluate(X_test, y_test)

592/592 [==============================] - 0s 420us/step - loss: 0.6671 - mae: 0.6508


In [93]:
preds = my_model.predict(X_test)

In [94]:
y_test

73984   -1.367675
73373   -1.058085
72588    1.313449
21818    0.655472
83635   -0.877848
           ...   
82350   -0.510633
30474    0.549762
24254   -0.746922
51831   -0.372112
33933    0.205977
Name: z-score, Length: 18930, dtype: float64

In [95]:
preds

array([[-0.5716591 ],
       [-0.1911489 ],
       [ 0.52797294],
       ...,
       [-0.12010144],
       [-0.13900071],
       [ 0.04361013]], dtype=float32)

In [67]:
en_df[en_df["wmdist"]== np.inf]

,source,z-score,pt,ref,1gram-precision,2gram-precision,3gram-precision,4gram-precision,1gram-recall,2gram-recall,3gram-recall,4gram-recall,wmdist,pt_len,ref_len
13077,34) das Halbfinale des olympischen Tennisturni...,-3.581507,[],"[the, semi, -, final, of, the, olympic, tennis...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,10
14887,34) das Halbfinale des olympischen Tennisturni...,-2.217306,[],"[the, semi, -, final, of, the, olympic, tennis...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,10
17883,34) das Halbfinale des olympischen Tennisturni...,-2.328265,[],"[34, clear, the, semi, -, finals, of, the, oly...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,11
19744,34) das Halbfinale des olympischen Tennisturni...,0.216676,[],"[the, semi, -, finals, of, the, olympic, tenni...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,9
21522,34) das Halbfinale des olympischen Tennisturni...,0.546947,[],"[34, the, semi, -, finals, of, the, olympic, t...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,10
23342,34) das Halbfinale des olympischen Tennisturni...,0.639098,[],"[34, the, semi, -, final, of, the, olympic, te...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,12
25451,34) das Halbfinale des olympischen Tennisturni...,0.018193,[],"[the, semi, -, final, of, the, olympic, tourna...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0,8
39524,Как и Бэм.,-0.540447,"[so, is, bam]",[],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,3,0
40120,Коу поморщился.,-6.428480,"[mr., coe, grimaced]",[],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,3,0
41170,Написал Ланцман.,-1.353824,"[lanzmann, wrote]",[],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,2,0
